<a href="https://colab.research.google.com/github/AbhishekAjoy/ML-DL/blob/master/BitcoinStock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator

Using TensorFlow backend.


In [2]:
filename = 'BTC-USD.csv'
df = pd.read_csv(filename)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 7 columns):
Date         1826 non-null object
Open         1826 non-null float64
High         1826 non-null float64
Low          1826 non-null float64
Close        1826 non-null float64
Adj Close    1826 non-null float64
Volume       1826 non-null int64
dtypes: float64(5), int64(1), object(1)
memory usage: 99.9+ KB


In [4]:
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-07-07,613.599976,619.650024,609.349976,615.320007,615.320007,2004899
1,2014-07-08,615.320007,623.820007,610.530029,623.000000,623.000000,1920482
2,2014-07-09,623.000000,624.950012,611.659973,618.390015,618.390015,2253171
3,2014-07-10,618.390015,633.619995,616.109985,632.900024,632.900024,1969199
4,2014-07-11,632.900024,638.590027,625.900024,635.900024,635.900024,2179286


In [5]:
df.set_axis(df['Date'],inplace=True)
df.drop(columns=['Open','High','Low','Volume'],inplace=True)
df.head()

,Date,Close,Adj Close
Date,,,
2014-07-07,2014-07-07,615.320007,615.320007
2014-07-08,2014-07-08,623.000000,623.000000
2014-07-09,2014-07-09,618.390015,618.390015
2014-07-10,2014-07-10,632.900024,632.900024
2014-07-11,2014-07-11,635.900024,635.900024


In [6]:
!pip install plotly==3.6.0
import plotly
import plotly.graph_objs as go
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
print("Plotly Version: ",plotly.__version__)

init_notebook_mode(connected=True)
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

     |████████████████████████████████| 31.1MB 36.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/67/0b/29/08c7f5caed2d1ac446db982ff607b326d49bfa0bd3a67ef8c7
Successfully built plotly
  Found existing installation: plotly 3.6.1
    Uninstalling plotly-3.6.1:
      Successfully uninstalled plotly-3.6.1
Plotly Version:  3.6.0


In [8]:
configure_plotly_browser_state()
trace = go.Scatter(x=df['Date'],
                   y=df['Close'],
                   mode = 'lines',
                   name='Data'
                  )
layout = go.Layout(title="",
                  xaxis = {'title' : "Date"},
                  yaxis = {'title' : "Close(Dollars)"}
                  )
fig = go.Figure(data=[trace], layout = layout)
iplot(fig)

In [9]:
close_data = df['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent * len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

data_train = df['Date'][:split]
data_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))


1460
366


In [0]:
look_back = 15

train_generator = TimeseriesGenerator(close_train,
                                      close_train,
                                      length = look_back,
                                      batch_size = 10
                                     )
test_generator = TimeseriesGenerator(close_test,
                                     close_test,
                                     length = look_back,
                                     batch_size = 1
                                    )


# Neural Network

In [15]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
          LSTM(10,
               activation ='relu',
               input_shape = (look_back,1))
          )
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse')

model.fit_generator(train_generator, epochs = 100, verbose = 1)

Epoch 1/100
145/145 [==============================] - 6s 39ms/step - loss: 21116895.8889
Epoch 2/100
145/145 [==============================] - 5s 34ms/step - loss: 17745415.8290
Epoch 3/100
145/145 [==============================] - 5s 34ms/step - loss: 7899868.3397
Epoch 4/100
145/145 [==============================] - 5s 34ms/step - loss: 5275114.4342
Epoch 5/100
145/145 [==============================] - 5s 34ms/step - loss: 3791556.4482
Epoch 6/100
145/145 [==============================] - 5s 34ms/step - loss: 1047043.4970
Epoch 7/100
145/145 [==============================] - 5s 34ms/step - loss: 334692.8526
Epoch 8/100
145/145 [==============================] - 5s 34ms/step - loss: 227828.1091
Epoch 9/100
145/145 [==============================] - 5s 34ms/step - loss: 223168.1120
Epoch 10/100
145/145 [==============================] - 5s 34ms/step - loss: 222945.3707
Epoch 11/100
145/145 [==============================] - 5s 34ms/step - loss: 219723.8263
Epoch 12/100
145/145 [

In [16]:
prediction = model.predict_generator(test_generator)
prediction

array([[ 7122.3516],
       [ 7148.599 ],
       [ 7334.815 ],
       [ 7770.3296],
       [ 7816.9575],
       [ 7725.092 ],
       [ 7842.193 ],
       [ 7919.445 ],
       [ 7944.2144],
       [ 7938.2583],
       [ 7714.905 ],
       [ 7595.159 ],
       [ 7532.196 ],
       [ 7442.316 ],
       [ 7199.406 ],
       [ 7132.7188],
       [ 7042.7627],
       [ 6873.4634],
       [ 6585.354 ],
       [ 6624.1675],
       [ 6389.9795],
       [ 6336.938 ],
       [ 6341.2524],
       [ 6269.002 ],
       [ 6169.87  ],
       [ 6143.361 ],
       [ 6137.82  ],
       [ 6285.2095],
       [ 6241.363 ],
       [ 6279.727 ],
       [ 6176.8076],
       [ 6259.379 ],
       [ 6219.61  ],
       [ 6296.8726],
       [ 6417.481 ],
       [ 6484.272 ],
       [ 6498.0347],
       [ 6613.6245],
       [ 6754.934 ],
       [ 6790.6685],
       [ 6783.154 ],
       [ 6803.481 ],
       [ 6909.5728],
       [ 7003.4565],
       [ 7024.9404],
       [ 7092.435 ],
       [ 6779.53  ],
       [ 6603

In [0]:
close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))


In [22]:
configure_plotly_browser_state()
trace1 = go.Scatter(x = data_train,
                    y = close_train,
                    mode = 'lines',
                    name = 'Data')
trace2 = go.Scatter(x = data_test,
                    y = prediction,
                    mode = 'lines',
                    name = 'Prediction')
trace3 = go.Scatter(x = data_test,
                    y = close_test,
                    mode = 'lines',
                    name = 'Ground Truth')
layout = go.Layout(title="BTC-USD",
                   xaxis = {'title' : "Date"},
                   yaxis = {'title' : "Close"})
fig = go.Figure(data=[trace1,trace2,trace3], layout = layout)
iplot(fig)